In [1]:
import sys
sys.path.append("/p300s/wangmx_group/xutingfeng/SIS/")


In [2]:

from sis.dataset import SISDataset
import torch 
import transformers 
from sis.model.sismodel import DoubleTransformerModel
from sis.utils import modelParametersNum, try_gpu

# Step1 dataset load 
# device = try_gpu()
device = "cpu"

sisdataset = SISDataset(root_dir="/p300s/wangmx_group/xutingfeng/SIS/sis/dataset/total_data.csv", device = device)

aa_vocab = sisdataset.aa_vocab
sis_datasetDict = sisdataset.dataset_dict

train_dataloader = torch.utils.data.DataLoader(sis_datasetDict["train"], batch_size=4)
test_dataloader = torch.utils.data.DataLoader(sis_datasetDict["test"], batch_size=4)




Using custom data configuration default-52200760334198e7
Reusing dataset csv (/home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-bec2fd355b0cb7d7.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-d7b3eee632703ea9.arrow


Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-fbd879c251d0f77b.arrow
Loading cached processed dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0f1986b4010acd7d.arrow
Loading cached split indices for dataset at /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-088d52191e3b3af4.arrow and /home/xutingfeng/.cache/huggingface/datasets/csv/default-52200760334198e7/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9b3b3c18e50420ce.arrow


In [5]:
from sis.train import train
from torch import nn 
from torch.utils.tensorboard import SummaryWriter
import os.path as osp
import os 

# Step2 build model
N = 4 
d_model = len(aa_vocab)
d_ff = 64 
dropout = 0.1 
length = sisdataset.SLF_max_length + sisdataset.SRnase_max_length

model = DoubleTransformerModel(N = N, d_model=d_model,d_ff=d_ff, dropout=dropout, seq_length=length) 

# Step3定义 optim and loss
optim = torch.optim.Adam(model.parameters(), lr=1e-4)
loss = nn.CrossEntropyLoss()
print(modelParametersNum(model))
## log path and save path
model_name = model.__class__.__name__
log_path = f"/p300s/wangmx_group/xutingfeng/SIS/output/logs/{model_name}"

summaryWriter = SummaryWriter(log_path)  # 使用tensorboard记录loss

save_path = f"/p300s/wangmx_group/xutingfeng/SIS/output/model/{model_name}"
# Step4 train
train(net = model, train_dataIter= train_dataloader, test_dataIter= test_dataloader,loss=loss, optim=optim, summary_writer = summaryWriter, epochs=100, device=device, savemodel=True, save_path = save_path)



模型总参数个数：40874
40874
epoch 10 : train: mean loss/per 0.808 acc is 0.780 and time is 18.845192926364565review/per sec
test: loss 1.728 and acc is 0.800
epoch 20 : train: mean loss/per 0.726 acc is 0.593 and time is 18.933578595251195review/per sec
test: loss 1.723 and acc is 0.533
epoch 30 : train: mean loss/per 0.661 acc is 0.729 and time is 18.4981399745578review/per sec
test: loss 1.777 and acc is 0.533
epoch 40 : train: mean loss/per 0.621 acc is 0.712 and time is 18.041212718232977review/per sec
test: loss 1.806 and acc is 0.533
